# 哈希表的基本操作
![](https://kingname-1257411235.cos.ap-chengdu.myqcloud.com/2019-03-03-16-52-34.png)


## 哈希表（Hash）

### 基本功能

Redis的Hash被设计出来，就是为了存储大量的键值对映射。储存相同数量的键值映射，Hash所占用的内存空间，远远小于字符串。

### 基本语法

```python
import redis

client = redis.Redis()

# 向Hash表中添加一个键值对
client.hset('Key', '字段名', '值')

# 向Hash表中添加多个键值对
client.hmset('Key', {'字段名1': '值1', '字段名2': '值2', '字段名3': '值3'})

# 查询字段是否在哈希表中
client.hexists('Key', '字段名')

# 查询哈希表中一个有多少个字段
client.hlen('Key')

# 获取Hash表里面所有的字段名（慎用）
client.hkeys('Key')

# 读取一个字段中的数据
client.hget('Key', '字段名')

# 读取多个字段中的数据
client.hmget('Key', ['字段名1', '字段名2', '字段名3'])

# 读取全部字段（慎用）
client.hgetall('Key')
```

## 向哈希表中写入数据

In [1]:
# 初始化Redis连接
import redis
import json
client = redis.Redis()

In [2]:
# 添加一条数据

info = json.dumps({'name': '张小二', 'age': 18, 'salary': 100, 'address': '北京'})
client.hset('user', 10001, info)

1

In [3]:
# 插入多条数据
info_dict = {
    10002: json.dumps({'name': '王小三', 'age': 27, 'salary': 10000, 'address': '浙江'}),
    10003: json.dumps({'name': '藏小四', 'age': 16, 'salary': 10, 'address': '四川'}),
    10004: json.dumps({'name': '刘小五', 'age': 30, 'salary': 990, 'address': '武汉'})
}
client.hmset('user', info_dict)

True

In [4]:
# 字段名不一定非要是数字，也可以是字母或者中文，字段值的数据类型也可以任意设定

client.hset('user', '马小七', 780)

1

## 检查字段信息

In [5]:
# 检查字段是否在Hash表中
client.hexists('user', 10003)

True

In [6]:
client.hexists('user', '马小七')

True

In [7]:
client.hexists('user', '不存在的字段')

False

In [8]:
client.hexists('不存在的Key', 10003)

False

In [9]:
# 查询Hash中有多少字段
client.hlen('user')

5

In [10]:
keys = client.hkeys('user')
for key in keys:
    print(key.decode())

马小七
10002
10003
10001
10004


## 获取键值对


In [11]:
# 获取单条数据
data = client.hget('user', '马小七')
print(f'返回的数据类型为：{type(data)}, 数据内容为：{data}')
print(f'数据解析以后为：{data.decode()}')


返回的数据类型为：<class 'bytes'>, 数据内容为：b'780'
数据解析以后为：780


In [12]:
data = client.hget('user', 10003)
data_dict = json.loads(data)
print(f'用JSON解析以后：{data_dict}')

用JSON解析以后：{'name': '藏小四', 'age': 16, 'salary': 10, 'address': '四川'}


In [13]:
# 批量获取数据

data_list = client.hmget('user', [10001, 10003])
for data in data_list:
    print(json.loads(data))

{'name': '张小二', 'age': 18, 'salary': 100, 'address': '北京'}
{'name': '藏小四', 'age': 16, 'salary': 10, 'address': '四川'}


In [14]:
## 获取全部数据

all_data = client.hgetall('user')
print(f'先来看看返回的数据是什么样的：{all_data}')



先来看看返回的数据是什么样的：{b'\xe9\xa9\xac\xe5\xb0\x8f\xe4\xb8\x83': b'780', b'10002': b'{"name": "\\u738b\\u5c0f\\u4e09", "age": 27, "salary": 10000, "address": "\\u6d59\\u6c5f"}', b'10003': b'{"name": "\\u85cf\\u5c0f\\u56db", "age": 16, "salary": 10, "address": "\\u56db\\u5ddd"}', b'10001': b'{"name": "\\u5f20\\u5c0f\\u4e8c", "age": 18, "salary": 100, "address": "\\u5317\\u4eac"}', b'10004': b'{"name": "\\u5218\\u5c0f\\u4e94", "age": 30, "salary": 990, "address": "\\u6b66\\u6c49"}'}


In [15]:
for field, value in all_data.items():
    print(f'字段名：{field.decode()}, 值：{json.loads(value)}')

字段名：马小七, 值：780
字段名：10002, 值：{'name': '王小三', 'age': 27, 'salary': 10000, 'address': '浙江'}
字段名：10003, 值：{'name': '藏小四', 'age': 16, 'salary': 10, 'address': '四川'}
字段名：10001, 值：{'name': '张小二', 'age': 18, 'salary': 100, 'address': '北京'}
字段名：10004, 值：{'name': '刘小五', 'age': 30, 'salary': 990, 'address': '武汉'}


![读者交流QQ群](https://kingname-1257411235.cos.ap-chengdu.myqcloud.com/2019-02-16-09-59-56.png)
![微信公众号](https://kingname-1257411235.cos.ap-chengdu.myqcloud.com/wechatplatform.jpg)
![](https://kingname-1257411235.cos.ap-chengdu.myqcloud.com/2019-03-03-20-47-47.png)

## 使用列表和Hash实现简单的任务队列
![](https://kingname-1257411235.cos.ap-chengdu.myqcloud.com/2019-03-03-16-52-34.png)


### 为什么需要任务队列？

有一系列比较耗时的任务，不能实时完成，此时就需要使用任务队列排队完成。例如，网站注册时需要发送验证邮件。发送1封邮件需要2秒钟。现在只有一台邮件服务器，有100人同时注册。

发邮件的过程不能让网站来完成。网站只是创建发邮件的任务，并把任务扔进任务队列中。另一个专门负责发邮件的程序从任务队列中读取任务，然后执行具体的发送操作。

### 任务队列需要实现哪些功能？

1. 添加任务
2. 删除任务
3. 暂停任务
4. 恢复被暂停的任务并重新排队

### 如何设计邮件的数据结构？

* 列表中存放任务ID
* Hash中存放任务详情，字段名为任务ID

任务详情的结构为：

```python
{
    "task_id": 123,
    "target": "xxx@163.com",
    "created_time": "2019-03-24 11:12:34"
}
```

## 添加任务

* 哈希表的Key为：task:detail
* 列表的Key为：task:queue

1. 首先创建任务详情，并写入Hash表中
2. 把任务ID写入到列表中

In [16]:
# 初始化Redis连接

import redis
client = redis.Redis()

In [17]:
# 添加任务
import datetime
import json


def add_task(task_id, target):
    task_detail = {'task_id': task_id,
                   'target': target,
                   'created_time': datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
    client.hset('task:detail', task_id, json.dumps(task_detail))
    client.rpush('task:queue', task_id)

In [18]:
# 发邮件的程序读取任务

def read_task():
    task_id = client.blpop('task:queue')[1].decode()
    task_detail = client.hget('task:detail', task_id)
    target = json.loads(task_detail)['target']
    print(f'给：{target} 发送邮件')
    

In [19]:
# 删除任务

def del_task(task_id):
    client.lrem('task:queue', 0, task_id)
    client.hdel('task:detail', task_id)

## 暂停和恢复任务

暂停和恢复任务都不会影响task:detail，只需要控制task:queue中的task_id即可。

### 暂停任务

1. 把task_id从task:queue中移除
2. 把task_id放入暂停列表：task:pause中

### 恢复任务

1. 把task_id从task:pause中移除
2. 把task_id重新放入task:queue右侧

In [20]:
# 暂停任务

def pause_task(task_id):
    client.lrem('task:queue', 0, task_id)
    client.rpush('task:pause', task_id)

In [21]:
# 恢复任务

def resume_task(task_id):
    client.lrem('task:pause', 0, task_id)
    client.rpush('task:queue', task_id)

## 来测试一下我们的简易任务队列

In [22]:
# 添加任务
add_task(1, 'contact@kingname.info')

In [23]:
add_task(2, 'register@163.com')

In [24]:
# 读取任务
read_task()

给：contact@kingname.info 发送邮件


In [25]:
add_task(3, 'rain@gmail.com')

In [26]:
add_task(4, 'world@hotmail.com')

In [27]:
## 删除任务
del_task(2)

In [28]:
add_task(5, 'hello@facebook.com')

In [29]:
pause_task(4)

In [30]:
read_task()

给：rain@gmail.com 发送邮件


In [31]:
read_task()

给：hello@facebook.com 发送邮件


In [32]:
# 恢复任务
resume_task(4)

In [33]:
read_task()

给：world@hotmail.com 发送邮件


![读者交流QQ群](https://kingname-1257411235.cos.ap-chengdu.myqcloud.com/2019-02-16-09-59-56.png)
![微信公众号](https://kingname-1257411235.cos.ap-chengdu.myqcloud.com/wechatplatform.jpg)
![](https://kingname-1257411235.cos.ap-chengdu.myqcloud.com/2019-03-03-20-47-47.png)

## 有序集合
![](https://kingname-1257411235.cos.ap-chengdu.myqcloud.com/2019-03-03-16-52-34.png)


## 有序集合（Sorted Set）

有序集合本质是`集合`，但是每一个元素都带有一个分数（Score），通过这个分数可以对元素进行排序。有序集合里面的元素不能重复，但分数可以重复。

### 常用命令

#### 添加数据

**注意！！最新版本的redis-py修改了参数的类型，请读者以视频为准。书上的写法可能会导致报错**

```python
client.zadd('有序集合Key', {'值1': 分数1, '值2': 分数2, '值3': 分数3})
```

#### 修改元素的评分

**redis-py也修改了这个方法的参数，请以视频为准，书上的写法在最新版的redis-py中会报错**
```python
client.zincrby('有序集合Key', 改变量, 集合中的元素)
```

#### 基于评分范围排序

```python

# 升序，注意第二个参数是下限，第三个参数是上限
client.zrangebyscore('有序集合Key', 评分下限, 评分上限, 切片起始位置, 切片数量, withscores=False)

# 降序，注意第二个参数是上限，第三个参数是下限
client.zrevrangebyscore('有序集合Key', 评分上限, 评分下限, 切片起始位置, 切片数量, withscores=False)
```

#### 基于位置排序

```python
client.zrange('有序集合Key', 开始位置（含）, 结束位置（含）, desc=False, withscores=False)
client.zrevrange('有序集合Key', 开始位置（含）, 结束位置（含）, withscores=False)
```

#### 查询值的排名

```python
client.zrank('有序集合Key', '值')
client.zrevrank('有序集合Key', '值')
```

#### 查询值的评分

```python
client.zscore('有序集合Key', '值')
```

#### 弹出最大最小值（Redis 5.0新特性）

```python
client.zpopmax('有序集合Key')
client.zpopmin('有序集合Key')
```

#### 统计元素个数

```python
# 统计所有元素的个数
client.zcard('有序集合Key')

# 统计某个评分分为内的元素个数
client.zcount('有序集合Key', 评分上限, 评分下限)
```

In [34]:
# 初始化连接
import redis

client = redis.Redis()

## 添加数据

## 特别说明：redis-py库的参数格式做了修改，书上的写法使用最新的redis-py库会导致报错。请以本视频的写法为准。

详情见：https://github.com/andymccurdy/redis-py#mset-msetnx-and-zadd

In [35]:
# 添加一条数据

client.zadd('gamerank', {'kingname': 10})

1

In [36]:
# 同时添加多条数据

client.zadd('gamerank', {'xiaoming': 8, 'Alice': 12, 'Ted': 7})

3

## 修改评分

## 注意：redis-py也修改了这个方法的参数，请以视频为准，书上的写法在最新版的redis-py中会报错

详情见：https://github.com/andymccurdy/redis-py#zincrby

In [37]:
# 把评分增加8

client.zincrby('gamerank', 8.0, 'kingname')

18.0

In [38]:
# 把评分减0.5

client.zincrby('gamerank', -0.5, 'Alice')

11.5

## 基于评分范围排序

In [39]:
# 补充一些数据进去

client.zadd('gamerank', {'Cine': 13, 'Jhon': 16, 'Susan': 15, 'Maxwell': 100})

4

In [40]:
# 升序排序，取前三个，不带分数

client.zrangebyscore('gamerank', 10, 20, 0, 3, withscores=False)

[b'Alice', b'Cine', b'Susan']

In [41]:
# 升序排序，取前三个，带分数

client.zrangebyscore('gamerank', 15, 100, 0, 3, withscores=True)

[(b'Susan', 15.0), (b'Jhon', 16.0), (b'kingname', 18.0)]

In [42]:
# 降序排序，取前三个，带分数

# 特别注意，这里的上限在前，下限在后
client.zrevrangebyscore('gamerank', 100, 15, 0, 3, withscores=True)

[(b'Maxwell', 100.0), (b'kingname', 18.0), (b'Jhon', 16.0)]

## 基于位置排序

In [43]:
# 升序排序
client.zrange('gamerank', 0, -1)

[b'Ted',
 b'xiaoming',
 b'Alice',
 b'Cine',
 b'Susan',
 b'Jhon',
 b'kingname',
 b'Maxwell']

In [44]:
# 升序排序，带分数

client.zrange('gamerank', 0, -1, withscores=True)

[(b'Ted', 7.0),
 (b'xiaoming', 8.0),
 (b'Alice', 11.5),
 (b'Cine', 13.0),
 (b'Susan', 15.0),
 (b'Jhon', 16.0),
 (b'kingname', 18.0),
 (b'Maxwell', 100.0)]

In [45]:
# 用升序排序的命令来实现降序排序

client.zrange('gamerank', 0, -1, desc=True, withscores=True)

[(b'Maxwell', 100.0),
 (b'kingname', 18.0),
 (b'Jhon', 16.0),
 (b'Susan', 15.0),
 (b'Cine', 13.0),
 (b'Alice', 11.5),
 (b'xiaoming', 8.0),
 (b'Ted', 7.0)]

In [46]:
# 用升序排序的命令来实现降序排序，但只取前3个

client.zrange('gamerank', 0, 2, desc=True, withscores=True)

[(b'Maxwell', 100.0), (b'kingname', 18.0), (b'Jhon', 16.0)]

In [47]:
# 升序排序，取最后3个数（也就是最大的三个数）
client.zrange('gamerank', -3, -1, withscores=True)

[(b'Jhon', 16.0), (b'kingname', 18.0), (b'Maxwell', 100.0)]

In [48]:
# 降序排序

client.zrevrange('gamerank', 0, -1)

[b'Maxwell',
 b'kingname',
 b'Jhon',
 b'Susan',
 b'Cine',
 b'Alice',
 b'xiaoming',
 b'Ted']

In [49]:
# 降序排序，带分数
client.zrevrange('gamerank', 0, -1, withscores=True)

[(b'Maxwell', 100.0),
 (b'kingname', 18.0),
 (b'Jhon', 16.0),
 (b'Susan', 15.0),
 (b'Cine', 13.0),
 (b'Alice', 11.5),
 (b'xiaoming', 8.0),
 (b'Ted', 7.0)]

In [50]:
# 降序排序，取前三个
client.zrevrange('gamerank', 0, 2, withscores=True)

[(b'Maxwell', 100.0), (b'kingname', 18.0), (b'Jhon', 16.0)]

## 查询值的排名

In [51]:
# 查询升序排名(注意，第一名对应的序号为0)
client.zrank('gamerank', 'kingname')

6

In [52]:
# 查询降序排名（注意，第一名对应的序号为0）
client.zrevrank('gamerank', 'kingname')

1

# 查询值的评分



In [53]:
client.zscore('gamerank', 'kingname')

18.0

# 弹出最值

In [54]:
# 弹出最大值

client.zpopmax('gamerank')

[(b'Maxwell', 100.0)]

In [55]:
# 弹出最小值

client.zpopmin('gamerank')

[(b'Ted', 7.0)]

## 统计元素格式


In [56]:
# 统计所有元素的个数

client.zcard('gamerank')

6

In [57]:
# 统计分数范围内的元素个数
client.zcount('gamerank', 15, 20)

3

![读者交流QQ群](https://kingname-1257411235.cos.ap-chengdu.myqcloud.com/2019-02-16-09-59-56.png)
![微信公众号](https://kingname-1257411235.cos.ap-chengdu.myqcloud.com/wechatplatform.jpg)
![](https://kingname-1257411235.cos.ap-chengdu.myqcloud.com/2019-03-03-20-47-47.png)

## 使用有序集合实现优先级队列和定时队列

![](https://kingname-1257411235.cos.ap-chengdu.myqcloud.com/2019-03-03-16-52-34.png)


## 优先级队列

玩过网游的读者可能会有这样的经验，有一些网游在开服前几天特别火爆，此时服务器压力太大，必需要排队进入游戏。如果玩家没有氪金，那么可能要排好几个小时才能进入游戏，但如果充值了会员，可能只需要几分钟就能进入游戏。

这就是一个优先级队列的应用。普通玩家的优先级低，VIP玩家优先级高，新的VIP玩家上线时能够直接插队到普通玩家的前面。可能有一些玩家一开始没有氪金，看到要排几个小时，于是充值了会员。此时优先级队列会自动修改玩家的优先级，把它排到前面去。

使用Redis的有序集合就能够实现一个简单的优先级队列。在有序集合里面，玩家的ID就是有序集合的值，玩家的优先级就是元素的评分`score`。使用`zpopmax`就可以每一次弹出评分最高的玩家ID。通过`zincrby`就可以修改玩家的评分，从而改动优先级。

In [108]:
# 添加初始数据

client = redis.Redis()

vip = {
    'kingname': 1000,
    'xiaoming': 800,
    'Alice':800
}

# 在实际项目中，为了防止评分相同的时候，元素不是按照上线顺序排序的情况，
# 这里可以使用时间戳作为评分，由于时间戳是不停增加的，所以必然先上线的玩家
# 时间戳小。对于VIP玩家，你可以在上线时间戳的基础上减去一个数。每次取zpopmin
# 这样就能实现先看优先级，再看上线时间的功能了。
normal = {
    'one': 1,
    'two': 1,
    'three': 1
}

client.zadd('enter_game', vip)
client.zadd('enter_game', normal)

3

In [109]:
# 先让score最大的玩家进入游戏

player = client.zpopmax('enter_game')
print(player)

[(b'kingname', 1000.0)]


In [110]:
# 突然来了一个新的VIP
client.zadd('enter_game', {'pm': 900})

1

In [111]:
# 再让当前优先级最高的玩家进入游戏

player = client.zpopmax('enter_game')
print(player)

[(b'pm', 900.0)]


In [112]:
# 普通玩家充值成为VIP

client.zincrby('enter_game', 500, 'two')

501.0

In [113]:
# 由于充值不够，他还是排在另外两个VIP的后面

player = client.zpopmax('enter_game')
print(player)

[(b'xiaoming', 800.0)]


In [114]:
player = client.zpopmax('enter_game')
print(player)

[(b'Alice', 800.0)]


In [115]:
# 终于轮到他了
player = client.zpopmax('enter_game')
print(player)

[(b'two', 501.0)]


## 定时任务队列

时间戳是一个整数部分10位数的浮点数，有序集合的评分也可以是浮点数，所以如果把Score设置为任务开始时间的时间戳，然后使用`zrangebyscore`每分钟查询一次有序集合，就可以实现精确到分钟的定时任务队列。

In [116]:
def time_str_2_timestamp(time_str):
    time_obj = datetime.datetime.strptime(time_str, '%Y-%m-%d %H:%M:%S')
    timestamp = time_obj.timestamp()
    return timestamp

In [117]:
# 创建任务
import datetime

task_dict = {
    'task1': '2019-04-14 15:27:01',
    'task2': '2019-04-14 15:27:01',
    'task3': '2019-04-14 15:28:01',
    'task4': '2019-04-14 15:28:27',
    'task5': '2019-04-14 15:29:28',
}

for key in task_dict:
    time_str = task_dict[key]
    timestamp = time_str_2_timestamp(time_str)
    task_dict[key] = timestamp
    client.zadd('timed_task', {key: timestamp})

In [119]:
# 通过一个死循环一不停获取任务
import time
while True:
    # 获取当前时间
    now = datetime.datetime.now() 
    
    # 把当前时间的秒和微秒替换为0
    now_lower = now.replace(second=0, microsecond=0)
    
    # 下一分钟
    now_higher = now_lower + datetime.timedelta(minutes=1)
    
    print(f'查询起始时间：{now_lower}, 查询截至时间：{now_higher}')
    task_for_this_minute = client.zrangebyscore('timed_task',
                                                now_lower.timestamp(),
                                                now_higher.timestamp(),
                                               0,
                                               1000,
                                               withscores=True)
    print(f'当前需要执行的任务：{task_for_this_minute}')
    time.sleep(60)
    
    

查询起始时间：2019-04-14 15:25:00, 查询截至时间：2019-04-14 15:26:00
当前需要执行的任务：[]
查询起始时间：2019-04-14 15:26:00, 查询截至时间：2019-04-14 15:27:00
当前需要执行的任务：[]
查询起始时间：2019-04-14 15:27:00, 查询截至时间：2019-04-14 15:28:00
当前需要执行的任务：[(b'task1', 1555226821.0), (b'task2', 1555226821.0)]
查询起始时间：2019-04-14 15:28:00, 查询截至时间：2019-04-14 15:29:00
当前需要执行的任务：[(b'task3', 1555226881.0), (b'task4', 1555226907.0)]
查询起始时间：2019-04-14 15:29:00, 查询截至时间：2019-04-14 15:30:00
当前需要执行的任务：[(b'task5', 1555226968.0)]


KeyboardInterrupt: 

![读者交流QQ群](https://kingname-1257411235.cos.ap-chengdu.myqcloud.com/2019-02-16-09-59-56.png)
![微信公众号](https://kingname-1257411235.cos.ap-chengdu.myqcloud.com/wechatplatform.jpg)
![](https://kingname-1257411235.cos.ap-chengdu.myqcloud.com/2019-03-03-20-47-47.png)